In [1]:
import sys
import os
import numpy as np
instagram_api_folder = os.path.dirname(os.getcwd())
sys.path.insert(1, instagram_api_folder)

#Useful if editing python files and want to keep jupyter notebook running
import importlib
import utils.safe_api
importlib.reload(utils.safe_api)

from utils.safe_api import SafeClientExtended

In [2]:
test_usernames = ['ryanlopezzzz', #small size public account example
                  'ruining99', #small size public account example
                  'i_am_jorge_murillo', #private account example
                  'cuteanimalzzzz123' #API login account
                 ]
medium_test_usernames = ['julia.chesson'] #medium size public account
large_test_usernames = ['cute.animals'] #large size public account

# How to Log-In
After logging in once, re-running this code does not use additional API calls.

Additional Key Word Arguments:
* **api_call_wait_time_generator** - Determines how long the API should wait before making request. Should be a function that takes no inputs and outputs a time in seconds. Default is random between 3s and 4s.
* **max_api_calls_per_hour** - Maximum amount of api calls per hour for single account. Throws ApiLimitReachedException on our side (safe) if reached this amount. Default: 160.
* **retry_failed_call** - If set to True, instead of throwing error when API limit is reached, program will pause and resume when safe. Default: False

In [3]:
kwargs = {
    'api_call_wait_time_generator' : (lambda : np.random.uniform(low=3.0, high=4.0)),
    'max_api_calls_per_hour' : 160,
    'retry_failed_call' : False
}

api_username = 'cuteanimalzzzz123'
api_password = 'instabotOP'
api = SafeClientExtended(api_username, api_password, **kwargs) #don't need to pass in **kwargs

# How to get single user info
Getting a single user's info only requires 1 API call. You can use **api.username_info(username)** or **api.user_info(user_id)**

In [4]:
collected_user_info = [] #collect this data for later use
user_info_keys = [
    'username',
    'user_id',
    'full_name',
    'private_status',
    'profile_pic_url',
    'media_count',
    'follower_count',
    'following_count',
    'bio_text',
    'url_in_bio',
    'hashtag_following_count',
    'usertags_count'
]
for username in test_usernames+medium_test_usernames+large_test_usernames:
    print('USER INFO:')
    username_info = api.username_info(username)
    for key in user_info_keys:
        print(f"{key}: {username_info[key]}")
    print('-'*80)
    print()
    collected_user_info.append(username_info)

USER INFO:
username: ryanlopezzzz
user_id: 255272463
full_name: Ryan Lopez
private_status: False
profile_pic_url: https://scontent-lax3-1.cdninstagram.com/v/t51.2885-19/s150x150/71332398_2133305030311574_6318002106229850112_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=102&_nc_ohc=wxx5-Asp1nsAX_GpapC&edm=AKralEIBAAAA&ccb=7-4&oh=00_AT-V_CD8ssOkBP9x0pe777bsZo295OIZYbvYRWRhhVZHlA&oe=61E3E192&_nc_sid=5e3072
media_count: 14
follower_count: 633
following_count: 390
bio_text: 
url_in_bio: https://youtu.be/nsaj6X8RsO0
hashtag_following_count: 0
usertags_count: 105
--------------------------------------------------------------------------------

USER INFO:
username: ruining99
user_id: 2142436680
full_name: Ruining Zhang
private_status: False
profile_pic_url: https://scontent-lax3-2.cdninstagram.com/v/t51.2885-19/s150x150/270095174_638765060885629_7676217408302914859_n.jpg?_nc_ht=scontent-lax3-2.cdninstagram.com&_nc_cat=101&_nc_ohc=i5k-IUBSKNQAX_CWuw1&edm=AKralEIBAAAA&ccb=7-4&oh=00_AT9e1

# How to get follower / following list

Typically returns 100 usernames / user ids per API call (+1 API call to check the user is not private). Doesn't always return full list. You can use **api.user_followers(user_id)** and **api.user_following(user_id)**

Additional Key Word Arguments (Useful for paging through long list of results.):
* **begin_query_id** - Keeps track of where you left off in the list.
* **num_results_stop_api_at** - Stop making API calls once this number of results has been returned (or until reach end of list). This will not necessarily be the number of results returned.

Note: Ping's API gives errors when requesting any follower info from very popular accounts. I got errors on cute.animals (88,000 followers) and justinbeiber (215M followers).

In [5]:
#Getting results on regular accounts
for user_index, username in enumerate(test_usernames):
    print(f'Collecting data on user: {username} \n')
    user_id = collected_user_info[user_index]['user_id']
    try:
        follower_count = collected_user_info[user_index]['follower_count']
        user_followers, _ = api.user_followers(user_id)
        print(f'Total followers: {follower_count}')
        print(f'Total followers loaded: {len(user_followers)}')
        print(f'Peeking at first three elements: {user_followers[:3]} \n')
        following_count = collected_user_info[user_index]['following_count']
        user_following, _ = api.user_following(user_id)
        print(f'Total following: {following_count}')
        print(f'Total following loaded: {len(user_following)}')
        print(f'Peeking at first three elements: {user_following[:3]}')
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print('-'*80)
    print()


Total followers: 633
Total followers loaded: 622
Peeking at first three elements: [[50004795460, 'cuteanimalzzzz123'], [8478000530, 'luca.scharrer'], [39612202963, 'carconnell_']] 

Total following: 390
Total following loaded: 390
Peeking at first three elements: [[8478000530, 'luca.scharrer'], [39612202963, 'carconnell_'], [249745606, 'grace.v.corrigan']]
--------------------------------------------------------------------------------


Total followers: 381
Total followers loaded: 379
Peeking at first three elements: [[8478000530, 'luca.scharrer'], [255272463, 'ryanlopezzzz'], [51207545762, 'allyminju']] 

Total following: 376
Total following loaded: 376
Peeking at first three elements: [[255272463, 'ryanlopezzzz'], [8478000530, 'luca.scharrer'], [51207545762, 'allyminju']]
--------------------------------------------------------------------------------


Exception: ValueError, Message: Can't get this info from private account.
--------------------------------------------------------

In [6]:
#When getting follower info from a 'more' popular account, it may be useful to do this in different chunks.
username = medium_test_usernames[0]
user_id = collected_user_info[-2]['user_id']
print(f'Collecting data on user: {username}')
follower_count = collected_user_info[-1]['follower_count']
print(f'Total followers: {follower_count}')

first_user_followers, next_begin_query_id = api.user_followers(user_id, num_results_stop_api_at = 300)
print(f'Loaded the first {len(first_user_followers)} followers.')
more_user_followers, _ = api.user_followers(user_id, begin_query_id = next_begin_query_id, 
                                            num_results_stop_api_at = 300)
print(f'Loaded an additional {len(more_user_followers)} followers.')
print('Sometimes the two lists contain duplicate followers, so should remove these.')
user_followers_no_duplicates = []
for user in first_user_followers + more_user_followers:
    if user not in user_followers_no_duplicates:
        user_followers_no_duplicates.append(user)
print(f'Total unique followers loaded: {len(user_followers_no_duplicates)}')

Total followers: 87817
Loaded the first 392 followers.
Loaded an additional 393 followers.
Sometimes the two lists contain duplicate followers, so should remove these.
Total unique followers loaded: 784


# How to get media (post) info from a user
Typically returns 18 posts per API call (+1 API call to check the user is not private). You can use **api.user_feed(user_id)**.

Additional Key Word Arguments (Useful for paging through long list of results.):
* **begin_query_id** - Keeps track of where you left off in the list.
* **num_results_stop_api_at** - Stop making API calls once this number of results has been returned (or until reach end of list). This will not necessarily be the number of results returned.

In [7]:
#Getting results on regular accounts
for user_index, username in enumerate(test_usernames):
    print(f'Collecting data on user: {username} \n')
    user_id = collected_user_info[user_index]['user_id']
    try:
        media_count = collected_user_info[user_index]['media_count']
        user_feed, _ = api.user_feed(user_id)
        print(f'Total number of posts: {media_count}')
        print(f'Total number of posts loaded: {len(user_feed)}')
        print(f'Peeking at first post data: {user_feed[0]} \n')
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print('-'*80)
    print()


Total number of posts: 14
Total number of posts loaded: 14
Peeking at first post data: {'media_id': '2112726015003571292_255272463', 'username': 'ryanlopezzzz', 'user_id': 255272463, 'like_count': 156, 'comment_count': 7, 'time_posted': 1566076588, 'num_sliding_content': 4, 'media_type': 8, 'caption_text': 'Pink wall lives up to the hype #fotofieldtrip', 'media_urls': ['https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/e35/67083131_493918131397061_976073748018960648_n.jpg?se=8&_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=105&_nc_ohc=EBYKIqCHvAEAX8Bg4YT&edm=ABmJApABAAAA&ccb=7-4&ig_cache_key=MjExMjcyNjAxMTc1Njk0NjAzMw%3D%3D.2-ccb7-4&oh=00_AT8AiNNcMBhjb2kkoCOalohYFerfKyrzMwAYsq6B4jHy9w&oe=61E2A198&_nc_sid=6136e7', 'https://scontent-lax3-2.cdninstagram.com/v/t51.2885-15/e35/68965965_655095278311512_5384214181216759067_n.jpg?_nc_ht=scontent-lax3-2.cdninstagram.com&_nc_cat=100&_nc_ohc=cqmlUsx-IgIAX8UI_gE&edm=ABmJApABAAAA&ccb=7-4&ig_cache_key=MjExMjcyNjAxMTc0ODY4MzI5OA%3D%3D.2-ccb7-4

In [8]:
#Getting post info in different chunks
username = collected_user_info[-1]['username']
user_id = collected_user_info[-1]['user_id']
print(f'Collecting data on user: {username}')
media_count = collected_user_info[-1]['media_count']
print(f'Total number of posts: {media_count}')

first_user_feed, next_begin_query_id = api.user_feed(user_id, num_results_stop_api_at = 40)
print(f'Loaded the first {len(first_user_feed)} posts.')
more_user_feed, _ = api.user_feed(user_id, begin_query_id = next_begin_query_id, 
                                            num_results_stop_api_at = 40)
print(f'Loaded an additional {len(more_user_feed)} posts.')
collected_user_feed = first_user_feed + more_user_feed
print(f'Total posts loaded: {len(collected_user_feed)}')

Total number of posts: 344
Loaded the first 54 posts.
Loaded an additional 54 posts.
Total posts loaded: 108


# How to follow / like / comment

You can check Ping's API documentation for how to do this. It seems to be easy to get banned for using these features. Before commenting or liking, should check if these features are enabled on the post, which can be done through Ping's original API (but not wrapper class).